In [1]:
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-c1qh5hfa
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-c1qh5hfa
  Created wheel for transformers: filename=transformers-3.1.0-cp36-none-any.whl size=956964 sha256=f926f8cdacbae7d01a1c79d5854c90ee25eb726965c6d4b4e3107fa8027441a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-h893xj9o/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [4]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py

--2020-09-16 18:25:21--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11434 (11K) [text/plain]
Saving to: ‘run_language_modeling.py.1’

run_language_modeli 100%[===================>]  11.17K  --.-KB/s    in 0.001s  

2020-09-16 18:25:22 (10.1 MB/s) - ‘run_language_modeling.py.1’ saved [11434/11434]



In [5]:
% ls

'Capstone Coversheet.gdoc'
'Capstone Coversheet Version 1 For Assignment 1.gdoc'
'Capstone Proposal (Junior Year).gdoc'
'Dev Journal.gdoc'
 Models/
 None/
 output/
 run_language_modeling.py
 run_language_modeling.py.1
 runs/


In [6]:
# get dataset
datapath = "Models/Data"
train_path = datapath + "/train.txt"
valid_path = datapath + "/valid.txt"
output = "./output"

In [7]:
%%bash 
export DIRECTORY=Models/Data
ls $DIRECTORY

cached_lm_GPT2Tokenizer_1024_train.txt
cached_lm_GPT2Tokenizer_1024_train.txt.lock
cached_lm_GPT2Tokenizer_1024_valid.txt
cached_lm_GPT2Tokenizer_1024_valid.txt.lock
test.txt
train.txt
valid.txt


In [12]:
%%bash
export TRAIN_FILE=Models/Data/train.txt
export TEST_FILE=Models/Data/valid.txt
export MODEL_NAME=gpt2
export OUTPUT_DIR=output

# python3 run_language_modeling.py \
#     --output_dir=$OUTPUT_DIR \
#     --model_type=$MODEL_NAME \
#     --model_name_or_path=$MODEL_NAME \
#     --do_train \
#     --train_data_file=$TRAIN_FILE \
#     --do_eval \
#     --eval_data_file=$TEST_FILE \
#     --per_gpu_train_batch_size=1 \
#     --save_steps=-1 \
#     --num_train_epochs=2

python run_language_modeling.py \
    --output_dir=$OUTPUT_DIR \
    --model_type=gpt2 \
    --model_name_or_path=$MODEL_NAME \
    --do_train \
    --train_data_file=$TRAIN_FILE \
    --do_eval \
    --eval_data_file=$TEST_FILE \
    --per_gpu_train_batch_size=1 \
    --save_steps=-1 \
    --num_train_epochs=2

{'loss': 1.8501156005859376, 'learning_rate': 2.7436823104693144e-05, 'epoch': 0.9025270758122743, 'total_flos': 382279090176000, 'step': 500}
{'loss': 1.5567396240234375, 'learning_rate': 4.873646209386281e-06, 'epoch': 1.8050541516245486, 'total_flos': 764558180352000, 'step': 1000}
{'eval_loss': 1.430858912675277, 'epoch': 2.0, 'total_flos': 847130463830016, 'step': 1108}


2020-09-16 18:35:46.806957: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
09/16/2020 18:35:48 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
09/16/2020 18:35:48 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='OUTPUT_DIR', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=1, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Sep16_18-35-48_ed150489d577', logging_first_step=False, logging_steps=500, save_steps=-1, save_total_limit=None, no_cuda=False, s

In [22]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import numpy as np

OUTPUT_DIR = "./OUTPUT_DIR"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
                                        
def generate(input_str, length=250, n=5):
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in range(length):
      outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    return output_text

def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

generated_text = generate("This aparatus \n", n=5)
print(generated_text)

This aparatus 
(A) is the compound represented by the following Chemical Formula 4: ##STR00242## wherein R.sup.1 to R.sup.3 are the same or different as or different from each other, and each of the following Chemical Formula 4 groups are independently selected from group A, group B, group C, group D, group E, group F, group G, group H, group I, and group K. <EOS>
<BOS> 1.  A light emitting device comprising: a substrate;  a first electrode;  and a second electrode formed on the substrate, the first electrode having a first conductivity of a metal;  a second conductivity of a metal;  a second electrode formed on the substrate, the second electrode having a second conductivity of a metal;  a third electrode formed on the substrate, the third electrode having a third conductivity of a metal;  a fourth electrode formed on the substrate, the fourth electrode having a fourth conductivity of a metal;  and a light emitting layer formed on the second electrode and the second electrode, the lig

In [17]:
% ls OUTPUT_DIR

config.json          merges.txt               tokenizer_config.json
eval_results_lm.txt  pytorch_model.bin        training_args.bin
log_history.json     special_tokens_map.json  vocab.json
